# PyCaret

O PyCaret é uma biblioteca de aprendizado de máquina em Python que simplifica a construção, treinamento, ajuste e implementação de modelos de machine learning. Ele automatiza grande parte do processo de modelagem, permitindo que você crie modelos complexos com apenas algumas linhas de código.

Principais Funcionalidades do PyCaret:
Automatização do Fluxo de Trabalho de Machine Learning:

Pré-processamento de dados (limpeza, transformação, normalização).
Treinamento de múltiplos modelos com diferentes algoritmos.
Comparação de modelos usando métricas padrão.
Ajuste fino (tuning) dos hiperparâmetros.
Modelos Preditivos:

PyCaret suporta algoritmos de classificação, regressão, clustering, detecção de anomalias, e séries temporais.
Pipeline Integrado:

Todo o fluxo de trabalho é encapsulado em um pipeline, garantindo que as etapas de transformação de dados sejam consistentes durante a previsão e produção.
Interpretação de Modelos:

PyCaret fornece ferramentas para interpretar modelos, visualizar importância de variáveis e analisar predições.

In [ ]:
!pip install openpyxl

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.utils.validation import check_X_y
import numpy as np

# Função para carregar datasets
def load_datasets(directory='dataset'):
    datasets = {}
    
    # Verifica se o diretório existe
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    # Percorre todos os arquivos do diretório
    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            # Detecta delimitador automaticamente
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
                print(datasets[filename].head())  # Exibe as primeiras linhas do dataframe
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")
    
    print(f"{len(datasets)} arquivos carregados com sucesso.")
    return datasets

# Função para treinar e testar modelos de regressão com validação cruzada
def test_regression_models(data):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor()
    }
    
    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'Lasso': {'alpha': [0.1, 0.5, 1]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
        'GradientBoosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]}
    }
    
    results = {}
    
    for name, df in data.items():
        # Remove valores nulos
        df = df.dropna()
        
        # Verifica se há dados suficientes
        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue
        
        X = df.iloc[:, :-1]  # Todas as colunas menos a última
        y = df.iloc[:, -1]   # Última coluna como alvo
        
        try:
            # Validação do X e y
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        for model_name, model in models.items():
            grid = GridSearchCV(model, param_grid[model_name], cv=5, scoring='neg_mean_squared_error', error_score='raise')
            try:
                grid.fit(X_train, y_train)
                best_model = grid.best_estimator_

                # Validação cruzada
                mse_scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
                r2_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='r2')
                
                mse = np.mean(mse_scores)
                rmse = np.sqrt(mse)
                r2 = np.mean(r2_scores)

                results[f'{name}_{model_name}'] = {
                    'mse': mse,
                    'rmse': rmse,
                    'r2_score': r2,
                    'best_params': grid.best_params_
                }
                print(f"{name} - {model_name}: MSE = {mse:.4f}, RMSE = {rmse:.4f}, R² = {r2:.4f}")
            except Exception as e:
                print(f"{name} - {model_name}: Falha no treinamento - {e}")
    
    # Salva resultados em um arquivo Excel
    results_df = pd.DataFrame.from_dict(results, orient='index')
    results_df.to_excel('regression_results.xlsx')
    print("Resultados salvos em 'regression_results.xlsx'.")
    
    return results

# Exemplo de uso
data = load_datasets()
results = test_regression_models(data)
for model, result in results.items():
    print(f"{model}: {result}")
